# Keywords AI Prompt Management Workflow Demo

This notebook demonstrates the complete workflow for managing prompts in Keywords AI, including creation, versioning, deployment, and usage with OpenAI SDK.

## What are Prompts in Keywords AI?

Keywords AI prompts are reusable templates that define AI model configurations, system messages, and parameters. They enable version control, A/B testing, and centralized prompt management. Each prompt can have multiple versions with different settings, allowing you to iterate and deploy the best-performing configurations. Prompts integrate seamlessly with OpenAI SDK through Keywords AI's gateway, providing analytics and monitoring for all your AI interactions.

## Prerequisites

- Keywords AI API key (`KEYWORDSAI_API_KEY`)
- Keywords AI base URL (`KEYWORDSAI_BASE_URL`) 
- Python packages: `keywordsai`, `openai`, `python-dotenv`


In [1]:
# Setup and imports
import os
from datetime import datetime, timezone
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Keywords AI imports
from keywordsai.prompts.api import PromptAPI
from keywordsai_sdk.keywordsai_types.prompt_types import Prompt, PromptVersion

# Check environment variables
api_key = os.getenv("KEYWORDSAI_API_KEY")
base_url = os.getenv("KEYWORDSAI_BASE_URL")

if not api_key or not base_url:
    print("❌ Missing environment variables!")
    print("Please set KEYWORDSAI_API_KEY and KEYWORDSAI_BASE_URL in your .env file")
else:
    print("✅ Environment variables loaded successfully")
    print(f"🔑 API Key: {api_key[:10]}...{api_key[-4:] if len(api_key) > 14 else api_key}")
    print(f"🌐 Base URL: {base_url}")

# Initialize the Prompt API client
client = PromptAPI(api_key=api_key, base_url=base_url)


✅ Environment variables loaded successfully
🔑 API Key: h5vo0ju3.n...2BzH
🌐 Base URL: http://localhost:8000


## Step 1: Create a New Prompt

Let's start by creating a new prompt that will serve as our container for different versions.


In [2]:
# Create a new prompt
print("📝 Creating a new prompt...")

# Create basic prompt
prompt = await client.acreate()
print(f"✅ Created prompt with ID: {prompt.prompt_id}")
print(f"   Name: {prompt.name}")
print(f"   Description: {prompt.description}")


📝 Creating a new prompt...
✅ Created prompt with ID: 63cc66077f3b4d349ea191d981647981
   Name: Untitled
   Description: 


In [3]:
# Update the prompt with meaningful information
print("\n✏️  Updating prompt metadata...")
updated_prompt_data = Prompt(
    name="Customer Support Assistant",
    description="AI assistant for handling customer support inquiries with professional tone",
    prompt_id=prompt.prompt_id,  # Include the required prompt_id
)

updated_prompt = await client.aupdate(prompt.prompt_id, updated_prompt_data)
print(f"✅ Updated prompt: {updated_prompt.name}")
print(f"📋 Description: {updated_prompt.description}")

# Store the prompt ID for later use
PROMPT_ID = updated_prompt.prompt_id
print(f"\n🔗 Prompt ID for reference: {PROMPT_ID}")
print(f"Prompt JSON representation: {updated_prompt.model_dump_json(indent=2)}")


✏️  Updating prompt metadata...
✅ Updated prompt: Customer Support Assistant
📋 Description: AI assistant for handling customer support inquiries with professional tone

🔗 Prompt ID for reference: 63cc66077f3b4d349ea191d981647981
Prompt JSON representation: {
  "id": "63cc66077f3b4d349ea191d981647981",
  "name": "Customer Support Assistant",
  "description": "AI assistant for handling customer support inquiries with professional tone",
  "prompt_id": "63cc66077f3b4d349ea191d981647981",
  "prompt_slug": "",
  "full_prompt_id": null,
  "current_version": null,
  "live_version": null,
  "prompt_versions": null,
  "prompt_activities": null,
  "commit_count": 0,
  "starred": false,
  "tags": [],
  "is_example": false,
  "blurred": false
}


In [4]:
# List all prompts to show the newly created one
print("📋 Listing all prompts to showcase the list feature...")
prompts_list = await client.alist(page_size=10)
print(f"✅ Found {prompts_list.count} total prompts")
print(f"📊 Showing first {len(prompts_list.results)} prompts:")

for i, p in enumerate(prompts_list.results[:5], 1):  # Show first 5
    is_our_prompt = "👈 Our new prompt!" if p.prompt_id == PROMPT_ID else ""
    print(f"      Created: {p.model_dump_json(indent=2)}")
    print()

📋 Listing all prompts to showcase the list feature...
✅ Found 2 total prompts
📊 Showing first 2 prompts:
      Created: {
  "id": "63cc66077f3b4d349ea191d981647981",
  "name": "Customer Support Assistant",
  "description": "AI assistant for handling customer support inquiries with professional tone",
  "prompt_id": "63cc66077f3b4d349ea191d981647981",
  "prompt_slug": "",
  "full_prompt_id": "63cc66077f3b4d349ea191d981647981",
  "current_version": null,
  "live_version": null,
  "prompt_versions": null,
  "prompt_activities": null,
  "commit_count": 0,
  "starred": false,
  "tags": [],
  "prompt_live_version_number": null
}

      Created: {
  "id": "6caa11b48d4d440986b3eb3b96ae795e",
  "name": "Test JSON schema",
  "description": "",
  "prompt_id": "6caa11b48d4d440986b3eb3b96ae795e",
  "prompt_slug": "something",
  "full_prompt_id": "6caa11b48d4d440986b3eb3b96ae795e",
  "current_version": {
    "id": "6caa11b48d4d440986b3eb3b96ae795e-31",
    "prompt_version_id": "6caa11b48d4d440986b3e

## 🌐 Browse the Keywords AI UI

**Now is a good time to visit the Keywords AI platform UI!**

1. Go to your Keywords AI dashboard: `https://platform.keywordsai.co`
2. Navigate to the **Prompts** section
3. Find the prompt you just created: "Customer Support Assistant"
4. Notice that it currently has no versions - you'll need to create one!

The UI provides a visual interface for managing prompts, but we'll continue with the API to show the complete programmatic workflow.


## Step 3: Create a Prompt Version

Now let's create a version of our prompt with specific configuration, messages, and model parameters.


In [ ]:
# Create a prompt version with specific configuration
print("🔧 Creating a prompt version...")

version_data = PromptVersion(
    prompt_version_id="version-001",
    description="Initial customer support version with professional tone",
    created_at=datetime.now(timezone.utc),
    updated_at=datetime.now(timezone.utc),
    version=1,
    messages=[
        {
            "role": "system",
            "content": "You are a professional customer support assistant. Always be helpful, polite, and provide clear solutions to customer problems.",
        },
        {"role": "user", "content": "{{customer_inquiry}}"},
    ],
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=2048,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    variables={"customer_inquiry": "Customer's question or issue"},
    parent_prompt=PROMPT_ID,
)

version = await client.acreate_version(PROMPT_ID, version_data)
print(f"✅ Created version {version.version}")
print(f"   Model: {version.model}")
print(f"   Temperature: {version.temperature}")
print(f"   Messages: {len(version.messages)} messages")
print(f"   Variables: {version.variables}")

# Store version info for later use
VERSION_1 = version.version
print(f"\n🔗 Version {VERSION_1} created successfully")
print(f"   Version detail: {version.model_dump_json(indent=2)}")


🔧 Creating a prompt version...
✅ Created version 1
   Model: gpt-4o-mini
   Temperature: 0.7
   Messages: 2 messages
   Variables: {'customer_inquiry': "Customer's question or issue"}

🔗 Version 1 created successfully
   Version detail: {"id":"63cc66077f3b4d349ea191d981647981-1","prompt_version_id":"63cc66077f3b4d349ea191d981647981-1","description":"Initial customer support version with professional tone","created_at":"2025-08-21T01:43:02.242817Z","updated_at":"2025-08-21T01:43:02.310866Z","version":1,"messages":[{"role":"system","content":"You are a professional customer support assistant. Always be helpful, polite, and provide clear solutions to customer problems.","name":null,"tool_call_id":null,"tool_calls":null,"reasoning_content":null,"thinking_blocks":null},{"role":"user","content":"{{customer_inquiry}}","name":null,"tool_call_id":null,"tool_calls":null,"reasoning_content":null,"thinking_blocks":null}],"edited_by":{"first_name":"Keywords AI","last_name":"Team","email":"admin@key

## Step 4: Update a Prompt Version

Let's update our version to demonstrate how you can modify existing versions.


In [6]:
# First, let's check if the version is readonly
retrieved_version = await client.aget_version(PROMPT_ID, VERSION_1)
print(f"🔍 Version {retrieved_version.version} details:")
print(f"   Description: {retrieved_version.description}")
print(f"   Readonly: {retrieved_version.readonly}")

if retrieved_version.readonly:
    print(f"\n⚠️  Version {retrieved_version.version} is readonly - cannot update")
    print("   This is expected behavior - versions may become readonly after creation")
else:
    print(f"\n✏️  Updating version {VERSION_1}...")
    # Only send the fields we want to update, not a full PromptVersion object
    update_version_data = {
        "description": "Updated: Professional customer support with improved clarity",
        "temperature": 0.6,  # Slightly lower temperature
        "max_tokens": 2500,  # More tokens
    }
    
    updated_version = await client.aupdate_version(
        PROMPT_ID, VERSION_1, update_version_data
    )
    print(f"✅ Updated version {updated_version.version}")
    print(f"   New description: {updated_version.description}")
    print(f"   New temperature: {updated_version.temperature}")
    print(f"   New max_tokens: {updated_version.max_tokens}")


🔍 Version 1 details:
   Description: Initial customer support version with professional tone
   Readonly: False

✏️  Updating version 1...
✅ Updated version 1
   New description: Updated: Professional customer support with improved clarity
   New temperature: 0.6
   New max_tokens: 2500


## Step 5: Setup OpenAI Client with Keywords AI

Now let's setup the OpenAI client to use Keywords AI as the gateway. This allows us to use our prompts with the standard OpenAI SDK.

Reference: [Keywords AI OpenAI SDK Integration](https://docs.keywordsai.co/integration/development-frameworks/llm_framework/openai/openai-sdk)


In [7]:
# Setup OpenAI client to use Keywords AI gateway
from openai import OpenAI

# Initialize OpenAI client with Keywords AI endpoint
openai_client = OpenAI(
    base_url=f"{base_url.rstrip('/')}/api/",  # Keywords AI gateway endpoint
    api_key=api_key,  # Use your Keywords AI API key
)

print("✅ OpenAI client configured to use Keywords AI gateway")
print(f"🔗 Gateway URL: {base_url.rstrip('/')}/api/")
print("\n💡 This allows you to:")
print("   - Use Keywords AI prompts with OpenAI SDK")
print("   - Access 250+ models through one interface")
print("   - Get detailed analytics and monitoring")
print("   - Manage fallbacks and error handling")


✅ OpenAI client configured to use Keywords AI gateway
🔗 Gateway URL: http://localhost:8000/api/

💡 This allows you to:
   - Use Keywords AI prompts with OpenAI SDK
   - Access 250+ models through one interface
   - Get detailed analytics and monitoring
   - Manage fallbacks and error handling


In [8]:
import json

# Define a reusable function for prompt-based chat completions
def chat_with_prompt(
    client, 
    prompt_id, 
    variables=None, 
    model="gpt-4o-mini",
    customer_identifier=None,
    metadata=None
):
    """
    Make a chat completion using a Keywords AI prompt.
    Shows the exact extra_body structure being sent for educational purposes.
    
    Args:
        client: OpenAI client configured for Keywords AI
        prompt_id: ID of the prompt to use
        variables: Dict of variables to substitute in the prompt
        model: Model to use (default: gpt-4o-mini)
        customer_identifier: Optional customer identifier for tracking
        metadata: Optional metadata dict for analytics
    
    Returns:
        OpenAI ChatCompletion response
    """
    # Build the correct extra_body structure
    extra_body = {
        "prompt": {
            "prompt_id": prompt_id,
        },
    }
    
    if variables:
        extra_body["prompt"]["variables"] = variables
    
    if customer_identifier:
        extra_body["customer_identifier"] = customer_identifier
        
    if metadata:
        extra_body["metadata"] = metadata
    
    # Print the structure so users can see exactly what's being sent
    print(f"🔧 Keywords AI Extra Body Structure:")
    print(json.dumps(extra_body, indent=2))
    print()
    
    return client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": ".",  # Placeholder for OAI SDK, will be replaced by the prompt
            }
        ],
        extra_body=extra_body,
    )

print("✅ Reusable chat_with_prompt function defined")
print("   • This function shows the correct extra_body structure")
print("   • Use this instead of manually building extra_body")


✅ Reusable chat_with_prompt function defined
   • This function shows the correct extra_body structure
   • Use this instead of manually building extra_body


## Step 6: Try to Use the Prompt (Expected Error)

Let's try to use our prompt with the OpenAI client. We expect this to fail because the prompt hasn't been deployed yet.


In [9]:
# Try to use the prompt - this should fail because it's not deployed
print("🧪 Attempting to use the prompt (expecting error)...")

try:
    response = chat_with_prompt(
        client=openai_client,
        prompt_id=PROMPT_ID,
        variables={
            "customer_inquiry": "My order hasn't arrived yet. What should I do?"
        },
        customer_identifier="demo_user_123",
        metadata={"demo": "prompt_workflow"}
    )
    
    print(f"✅ Unexpected success! Response: {response.choices[0].message.content}")
    
except Exception as e:
    print(f"❌ Expected error occurred: {str(e)}")
    print(f"🔍 Error type: {type(e).__name__}")
    print("\n💡 This is expected because:")
    print("   - The prompt hasn't been deployed yet")
    print("   - Prompts need to be deployed before they can be used")
    print("   - We'll deploy it in the next step")


🧪 Attempting to use the prompt (expecting error)...
🔧 Keywords AI Extra Body Structure:
{
  "prompt": {
    "prompt_id": "63cc66077f3b4d349ea191d981647981",
    "variables": {
      "customer_inquiry": "My order hasn't arrived yet. What should I do?"
    }
  },
  "customer_identifier": "demo_user_123",
  "metadata": {
    "demo": "prompt_workflow"
  }
}

❌ Expected error occurred: Error code: 404 - {'error': {'code': 'keywordsai_error', 'message': 'No live version found for the prompt, try deploying a version first!', 'param': None, 'type': 'keywordsai_error'}}
🔍 Error type: NotFoundError

💡 This is expected because:
   - The prompt hasn't been deployed yet
   - Prompts need to be deployed before they can be used
   - We'll deploy it in the next step


## Step 7: Try to Deploy the Prompt (Expected Draft Error)

Let's try to deploy our prompt. This might fail if the prompt is still in draft state.


In [10]:
# Try to deploy the prompt - this should fail because we only have draft versions
print("🚀 Attempting to deploy the prompt (expecting failure)...")

try:
    # Try to deploy the prompt using the actual deploy method
    deployed_prompt = await client.adeploy(PROMPT_ID)
    print(f"✅ Unexpected success! Prompt deployed:")
    print(f"   Live version: {deployed_prompt.live_version.version if deployed_prompt.live_version else 'None'}")
    
except Exception as e:
    print(f"❌ Expected deployment failure: {str(e)}")
    print(f"🔍 Error type: {type(e).__name__}")
    print("\n💡 This is expected because:")
    print("   - We only have draft versions so far")
    print("   - Draft versions cannot be deployed")
    print("   - We need to commit the draft first (create another version)")
    
    # Let's verify the current state
    prompt_details = await client.aget(PROMPT_ID)
    print(f"\n📋 Current prompt state:")
    print(f"   Name: {prompt_details.name}")
    print(f"   Total versions: {prompt_details.commit_count}")
    print(f"   Live version: {prompt_details.live_version}")
    print("   → No live version means prompt is not deployed yet")


🚀 Attempting to deploy the prompt (expecting failure)...
✅ Unexpected success! Prompt deployed:
   Live version: None


## Step 8: Commit Draft Version

To make a version deployable, we need to commit the current draft. This creates a readonly version that can be deployed.


In [12]:
# Create a new draft version that commits the current draft into a readonly version
print("📦 Creating a new draft version to commit the current one...")

# This creates version 2, which makes version 1 readonly and deployable
version_data_v2 = PromptVersion(
    prompt_version_id="version-002",
    description="Second version - this commits version 1",
    created_at=datetime.now(timezone.utc),
    updated_at=datetime.now(timezone.utc),
    version=2,
    messages=[
        {
            "role": "system",
            "content": "You are a friendly customer support assistant. Use a warm, professional tone while solving customer problems efficiently.",
        },
        {"role": "user", "content": "{{customer_inquiry}}"},
    ],
    model="gpt-4o-mini",
    temperature=0.8,  # Slightly higher temperature for more creativity
    max_tokens=1800,
    top_p=0.95,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    variables={"customer_inquiry": "Customer's question or issue"},
    parent_prompt=PROMPT_ID,
)

version_v2 = await client.acreate_version(PROMPT_ID, version_data_v2)
print(f"✅ Created version {version_v2.version}")
print(f"   This should make version {VERSION_1} readonly and deployable")

# Check if version 1 is now readonly
version_1_check = await client.aget_version(PROMPT_ID, VERSION_1)
print(f"\n🔍 Version {VERSION_1} status:")
print(f"   Readonly: {version_1_check.readonly}")
print(f"   {'✅ Ready for deployment' if version_1_check.readonly else '⚠️  Still draft'}")

VERSION_2 = version_v2.version
print(f"\n🔗 Version {VERSION_2} created (current draft)")


📦 Creating a new draft version to commit the current one...
✅ Created version 2
   This should make version 1 readonly and deployable

🔍 Version 1 status:
   Readonly: True
   ✅ Ready for deployment

🔗 Version 2 created (current draft)


In [13]:
# Deploy the prompt to make it live
print("🚀 Deploying the prompt...")

try:
    deployed_prompt = await client.adeploy(PROMPT_ID)
    print(f"✅ Successfully deployed prompt!")
    print(f"   Live version: {deployed_prompt.live_version.version if deployed_prompt.live_version else 'None'}")
    print(f"   Prompt is now ready for use with OpenAI SDK")
    
    # Verify deployment by checking prompt details
    deployment_check = await client.aget(PROMPT_ID)
    if deployment_check.live_version:
        print(f"   🔍 Deployment confirmed - Live version {deployment_check.live_version.version} is active")
        print(f"   Model: {deployment_check.live_version.model}")
        print(f"   Temperature: {deployment_check.live_version.temperature}")
    else:
        print("   ⚠️  No live version found after deployment attempt")
        
except Exception as deploy_error:
    print(f"❌ Deployment failed: {str(deploy_error)}")
    print("💡 This might happen if:")
    print("   - No readonly version is available")
    print("   - All versions are still in draft state") 
    print("   - API deployment endpoint is not available")

print("\n💡 What deployment does:")
print("   - Makes a readonly version the 'live' version")
print("   - Enables the prompt for use with OpenAI SDK")
print("   - Allows variable substitution in prompt messages")
print("   - Provides analytics and monitoring through Keywords AI")


🚀 Deploying the prompt...
✅ Successfully deployed prompt!
   Live version: None
   Prompt is now ready for use with OpenAI SDK
   ⚠️  No live version found after deployment attempt

💡 What deployment does:
   - Makes a readonly version the 'live' version
   - Enables the prompt for use with OpenAI SDK
   - Allows variable substitution in prompt messages
   - Provides analytics and monitoring through Keywords AI


## Step 10: Test the Deployed Prompt

Now that we have deployed the prompt, let's test it with the OpenAI SDK to verify it works correctly.


In [15]:
# Now try to use the prompt with OpenAI client
print("🚀 Attempting to use the prompt with OpenAI client...")

try:
    # Use the reusable function to make a proper prompt-based call
    response = chat_with_prompt(
        client=openai_client,
        prompt_id=PROMPT_ID,
        variables={
            "customer_inquiry": "My order hasn't arrived yet. What should I do?"
        },
        customer_identifier="demo_user_123",
        metadata={
            "demo": "prompt_workflow",
            "prompt_used": PROMPT_ID,
            "version_used": VERSION_1
        }
    )
    
    print("✅ Success! Response received:")
    print(f"📝 Content: {response.choices[0].message.content}")
    print(f"🔧 Model used: {response.model}")
    print(f"📊 Tokens used: {response.usage.total_tokens if response.usage else 'N/A'}")
    
    print("\n💡 What happened:")
    print("   - Request went through Keywords AI gateway")
    print("   - Used the committed prompt version")
    print("   - Got analytics and monitoring")
    print("   - Response logged in Keywords AI dashboard")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")
    print(f"🔍 Error type: {type(e).__name__}")
    print("\n💡 If this failed, it might be because:")
    print("   - Prompt deployment is not fully implemented yet")
    print("   - Need to manually deploy in the UI")
    print("   - API endpoint differences")


🚀 Attempting to use the prompt with OpenAI client...
🔧 Keywords AI Extra Body Structure:
{
  "prompt": {
    "prompt_id": "63cc66077f3b4d349ea191d981647981",
    "variables": {
      "customer_inquiry": "My order hasn't arrived yet. What should I do?"
    }
  },
  "customer_identifier": "demo_user_123",
  "metadata": {
    "demo": "prompt_workflow",
    "prompt_used": "63cc66077f3b4d349ea191d981647981",
    "version_used": 1
  }
}

✅ Success! Response received:
📝 Content: Hello! How can I assist you today?
🔧 Model used: gpt-4o-mini-2024-07-18
📊 Tokens used: 17

💡 What happened:
   - Request went through Keywords AI gateway
   - Used the committed prompt version
   - Got analytics and monitoring
   - Response logged in Keywords AI dashboard


## Step 11: Iterate - Create a New Version

Let's create a new version with different settings to demonstrate iteration and A/B testing capabilities.


In [16]:
# Create a new version with different approach - more concise and direct
print("🔧 Creating version 3 with a different approach...")

version_data_v3 = PromptVersion(
    prompt_version_id="version-003",
    description="Concise and direct customer support version",
    created_at=datetime.now(timezone.utc),
    updated_at=datetime.now(timezone.utc),
    version=3,
    messages=[
        {
            "role": "system",
            "content": "You are a concise customer support assistant. Provide direct, actionable solutions quickly. Be helpful but brief.",
        },
        {"role": "user", "content": "{{customer_inquiry}}"},
    ],
    model="gpt-4o-mini",
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=1000,  # Shorter responses
    top_p=0.8,
    frequency_penalty=0.2,
    presence_penalty=0.0,
    variables={"customer_inquiry": "Customer's question or issue"},
    parent_prompt=PROMPT_ID,
)

version_v3 = await client.acreate_version(PROMPT_ID, version_data_v3)
print(f"✅ Created version {version_v3.version}")
print(f"   Model: {version_v3.model}")
print(f"   Temperature: {version_v3.temperature} (lower for more focused responses)")
print(f"   Max tokens: {version_v3.max_tokens} (shorter responses)")

# Check version 2 status (should now be readonly)
version_2_check = await client.aget_version(PROMPT_ID, VERSION_2)
print(f"\n🔍 Version {VERSION_2} status:")
print(f"   Readonly: {version_2_check.readonly}")
print(f"   {'✅ Ready for deployment' if version_2_check.readonly else '⚠️  Still draft'}")

VERSION_3 = version_v3.version
print(f"\n🔗 Version {VERSION_3} created (current draft)")

# List all versions
print(f"\n📋 All versions for prompt {PROMPT_ID}:")
versions_list = await client.alist_versions(PROMPT_ID)
for v in versions_list.results:
    status = "readonly" if v.readonly else "draft"
    print(f"   - Version {v.version}: {v.description} ({status})")
    print(f"     Temperature: {v.temperature}, Max tokens: {v.max_tokens}")


🔧 Creating version 3 with a different approach...
✅ Created version 3
   Model: gpt-4o-mini
   Temperature: 0.3 (lower for more focused responses)
   Max tokens: 1000 (shorter responses)

🔍 Version 2 status:
   Readonly: True
   ✅ Ready for deployment

🔗 Version 3 created (current draft)

📋 All versions for prompt 63cc66077f3b4d349ea191d981647981:
   - Version 3: Concise and direct customer support version (draft)
     Temperature: 0.3, Max tokens: 1000
   - Version 2: Second version - this commits version 1 (readonly)
     Temperature: 0.8, Max tokens: 1800
   - Version 1: Updated: Professional customer support with improved clarity (readonly)
     Temperature: 0.6, Max tokens: 2500


## Step 12: Deploy New Version

Let's test the different versions to see how they perform with the same input.


In [17]:
# Compare different versions with the same input
test_message = "My order hasn't arrived yet and I need it urgently for a meeting tomorrow. What can you do to help?"

print("🧪 Testing different versions with the same input:")
print(f"📝 Test message: {test_message}")
print("\n" + "="*80)

# Test Version 1 (Professional tone, temp 0.7)
print("\n🔍 Testing Version 1 (Professional, temp=0.7):")
try:
    response_v1 = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a professional customer support assistant. Always be helpful, polite, and provide clear solutions to customer problems."
            },
            {"role": "user", "content": test_message}
        ],
        temperature=0.7,
        max_tokens=2048,
        extra_body={
            "customer_identifier": "demo_user_123",
            "metadata": {"version_test": "v1", "demo": "comparison"}
        }
    )
    print(f"✅ Version 1 Response: {response_v1.choices[0].message.content[:200]}...")
    print(f"📊 Tokens: {response_v1.usage.total_tokens if response_v1.usage else 'N/A'}")
except Exception as e:
    print(f"❌ Version 1 Error: {str(e)}")

print("\n" + "-"*50)

# Test Version 2 (Friendly tone, temp 0.8) 
print("\n🔍 Testing Version 2 (Friendly, temp=0.8):")
try:
    response_v2 = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a friendly customer support assistant. Use a warm, professional tone while solving customer problems efficiently."
            },
            {"role": "user", "content": test_message}
        ],
        temperature=0.8,
        max_tokens=1800,
        extra_body={
            "customer_identifier": "demo_user_123",
            "metadata": {"version_test": "v2", "demo": "comparison"}
        }
    )
    print(f"✅ Version 2 Response: {response_v2.choices[0].message.content[:200]}...")
    print(f"📊 Tokens: {response_v2.usage.total_tokens if response_v2.usage else 'N/A'}")
except Exception as e:
    print(f"❌ Version 2 Error: {str(e)}")

print("\n" + "-"*50)

# Test Version 3 (Concise, temp 0.3)
print("\n🔍 Testing Version 3 (Concise, temp=0.3):")
try:
    response_v3 = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a concise customer support assistant. Provide direct, actionable solutions quickly. Be helpful but brief."
            },
            {"role": "user", "content": test_message}
        ],
        temperature=0.3,
        max_tokens=1000,
        extra_body={
            "customer_identifier": "demo_user_123",
            "metadata": {"version_test": "v3", "demo": "comparison"}
        }
    )
    print(f"✅ Version 3 Response: {response_v3.choices[0].message.content[:200]}...")
    print(f"📊 Tokens: {response_v3.usage.total_tokens if response_v3.usage else 'N/A'}")
except Exception as e:
    print(f"❌ Version 3 Error: {str(e)}")

print("\n" + "="*80)
print("\n💡 Key Observations:")
print("   - Different versions produce different response styles")
print("   - Temperature affects creativity vs consistency")
print("   - Max tokens controls response length")
print("   - All requests are logged in Keywords AI for analysis")
print("   - You can A/B test different versions in production")


🧪 Testing different versions with the same input:
📝 Test message: My order hasn't arrived yet and I need it urgently for a meeting tomorrow. What can you do to help?


🔍 Testing Version 1 (Professional, temp=0.7):
✅ Version 1 Response: I’m sorry to hear that your order hasn’t arrived yet, especially since you need it for an important meeting. To assist you better, could you please provide me with your order number? I’ll check the st...
📊 Tokens: 115

--------------------------------------------------

🔍 Testing Version 2 (Friendly, temp=0.8):
✅ Version 2 Response: I completely understand how important it is to receive your order on time, especially with your meeting coming up tomorrow. Let’s get this sorted out for you!

Could you please provide me with your or...
📊 Tokens: 121

--------------------------------------------------

🔍 Testing Version 3 (Concise, temp=0.3):
✅ Version 3 Response: Please provide your order number, and I can check the status for you. If it's delayed, I can as